# Project Setup

In [3]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

import seaborn as sns
import time
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)


from talib.abstract import *
import talib


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression

from sklearn import preprocessing
from sklearn import decomposition

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score   
    
from sklearn.ensemble import VotingClassifier   
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier    
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from pytrends.request import TrendReq

from binance.client import Client
api_key = 'e0LdDWHVXTAdEAOPFT7r3Kgy4WX0iuyIsfKZRLFHStp2rnsTyddoYRBLHFfAzzi3'
api_secret = 'Ps0tL8vZJyUGFdDl6C5FbYnroSi4j0kOEYa2OlLGu5vZLx6NVyc1EsgJdlgZOApR'
client = Client(api_key, api_secret)

quandl.ApiConfig.api_key = 'UX1w8dNS5nXfuCqLQSx5'
data_dir = './data/'

In [11]:
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2017")

In [12]:
klines

[[1499990400000,
  '0.00375000',
  '0.00375000',
  '0.00235700',
  '0.00249900',
  '112991.60000000',
  1500076799999,
  '281.08244633',
  1643,
  '73474.85000000',
  '181.60204592',
  '166748.00000000'],
 [1500076800000,
  '0.00253600',
  '0.00261600',
  '0.00240600',
  '0.00251400',
  '197680.31000000',
  1500163199999,
  '496.27849280',
  4028,
  '107334.30000000',
  '270.40184527',
  '181360.00000000'],
 [1500163200000,
  '0.00251200',
  '0.00265000',
  '0.00245000',
  '0.00263800',
  '163064.45000000',
  1500249599999,
  '419.41072752',
  3302,
  '105633.69000000',
  '271.69305964',
  '218593.00000000'],
 [1500249600000,
  '0.00263800',
  '0.00274300',
  '0.00259100',
  '0.00266600',
  '154181.47000000',
  1500335999999,
  '410.05467232',
  3185,
  '96690.49000000',
  '257.13178635',
  '224842.00000000'],
 [1500336000000,
  '0.00266900',
  '0.00273100',
  '0.00256300',
  '0.00264300',
  '162311.49000000',
  1500422399999,
  '426.73412821',
  3350,
  '100907.27000000',
  '265.21926

In [2]:
! rm -r ./data
! mkdir ./data

## Pull BTC Exchange Pricing Data
The nature of Bitcoin exchanges is that the pricing is determined by supply and demand, hence no single exchange contains a true "master price" of Bitcoin.  To solve this issue, along with that of down-spikes, we'll pull data from three more major Bitcoin changes to calculate an aggregate Bitcoin price index.

In [3]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}{}.pkl'.format(data_dir, quandl_id.replace('/','-'))
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas", start_date="2010-01-01")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [4]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINBASE','BITSTAMP','ITBIT', 'KRAKEN']

exchange_data = {}

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

btc_usd_df = pd.DataFrame()

# Merge the BTC price dataseries' into a single dataframe
btc_usd_df['weightedAverage'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Weighted Price').replace(0,np.nan).mean(axis=1)  
btc_usd_df['quoteVolume'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Volume (Currency)') .replace(0, np.nan).mean(axis=1)
btc_usd_df['volume'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Volume (BTC)').replace(0, np.nan).mean(axis=1)
btc_usd_df['close'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Close').replace(0, np.nan).mean(axis=1)
btc_usd_df['low'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Low').replace(0, np.nan).mean(axis=1)
btc_usd_df['high'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'High').replace(0, np.nan).mean(axis=1)
btc_usd_df['open'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Open').replace(0, np.nan).mean(axis=1)

# Plot the average BTC price
btc_trace = go.Scatter(x=btc_usd_df.index, y=btc_usd_df['weightedAverage'])
py.iplot([btc_trace])


btc_usd_df.dropna(inplace=True)

btc_usd_df.head()

Cached BCHARTS/COINBASEUSD at ./data/BCHARTS-COINBASEUSD.pkl
Cached BCHARTS/BITSTAMPUSD at ./data/BCHARTS-BITSTAMPUSD.pkl
Cached BCHARTS/ITBITUSD at ./data/BCHARTS-ITBITUSD.pkl
Cached BCHARTS/KRAKENUSD at ./data/BCHARTS-KRAKENUSD.pkl


,weightedAverage,quoteVolume,volume,close,low,high,open
Date,,,,,,,
2011-09-13,5.929231,346.097389,58.371382,5.97,5.65,6.00,5.80
2011-09-14,5.590798,341.854813,61.145984,5.53,5.52,5.72,5.58
2011-09-15,5.094272,408.259002,80.140795,5.13,5.00,5.24,5.12
2011-09-16,4.854515,193.763147,39.914007,4.85,4.80,4.87,4.82
2011-09-17,4.870000,1.461000,0.300000,4.87,4.87,4.87,4.87


# Stock market OHLC data

In [5]:
visa_stock = get_quandl_data('EOD/V')[['Adj_Open', 'Adj_High', 'Adj_Low', 'Adj_Close', 'Adj_Volume']]

visa_stock = visa_stock.rename(columns={'Adj_Open': 'open'})
visa_stock = visa_stock.rename(columns={'Adj_High': 'high'})
visa_stock = visa_stock.rename(columns={'Adj_Low': 'low'})
visa_stock = visa_stock.rename(columns={'Adj_Close': 'close'})
visa_stock = visa_stock.rename(columns={'Adj_Volume': 'volume'})

visa_stock['weightedAverage'] = (visa_stock['low'] + visa_stock['high'])/2
visa_stock['quoteVolume'] = visa_stock['volume']

visa_stock.head()

Cached EOD/V at ./data/EOD-V.pkl


,open,high,low,close,volume,weightedAverage,quoteVolume
Date,,,,,,,
2010-01-04,20.697681,20.878786,20.570673,20.730609,20180000.0,20.724729,20180000.0
2010-01-05,20.533041,20.587137,20.227279,20.493056,25833600.0,20.407208,25833600.0
2010-01-06,20.464832,20.481296,20.203759,20.217871,16254000.0,20.342528,16254000.0
2010-01-07,20.220223,20.462480,20.121439,20.406032,27841200.0,20.291960,27841200.0
2010-01-08,20.420144,20.554209,20.281376,20.462480,11907200.0,20.417792,11907200.0


# Retrieve Altcoin Pricing Data
Now that we have a solid time series dataset for the price of Bitcoin, let's pull in some data on non-Bitcoin cryptocurrencies, commonly referred to as altcoins.

## Define Poloniex API Helper Functions

For retrieving data on cryptocurrencies we'll be using the [Poloniex API](https://poloniex.com/support/api/).  To assist in the altcoin data retrieval, we'll define two helper functions to download and cache JSON data from this API.

First, we'll define `get_json_data`, which will download and cache JSON data from a provided URL.

In [6]:
def get_json_data(json_url, file_name):
    '''Download and cache JSON data, return as a dataframe.'''
    cache_path = '{}{}.pkl'.format(data_dir, file_name)
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(file_name))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached response at {}'.format(cache_path))
    return df

Next, we'll define a function to format Poloniex API HTTP requests and call our new get_json_data function to save the resulting data.

In [7]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015

end_date = datetime.now() # up until today
period = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), period)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

This function will take a cryptocurrency pair string (such as 'BTC_ETH') and return the dataframe containing the historical exchange rate of the two currencies.

## Download Trading Data From Binance
Most altcoins cannot be bought directly with USD; to acquire these coins individuals often buy Bitcoins and then trade the Bitcoins for altcoins on cryptocurrency exchanges.  For this reason we'll be downloading the exchange rate to BTC for each coin, and then we'll use our existing BTC pricing data to convert this value to USD.

We'll download exchange data for nine of the top cryptocurrencies -
[Ethereum](https://www.ethereum.org/), [Litecoin](https://litecoin.org/), [Ripple](https://ripple.com/), [Ethereum Classic](https://ethereumclassic.github.io/), [Stellar](https://www.stellar.org/), [Dashcoin](http://dashcoin.info/), [Siacoin](http://sia.tech/), [Monero](https://getmonero.org/), and [NEM](https://www.nem.io/).

In [8]:
binance_coins = ['ETH', 'LTC', 'ETC','XRP', 'LSK', 'XMR',
                 'STRAT', 'ZEC', 'DASH', 'BTS', 'VIA', 'STEEM', 'NAV'] 

altcoin_data = {}
for altcoin in binance_coins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Cached response at ./data/BTC_ETH.pkl
Cached response at ./data/BTC_LTC.pkl
Cached response at ./data/BTC_ETC.pkl
Cached response at ./data/BTC_XRP.pkl
Cached response at ./data/BTC_LSK.pkl
Cached response at ./data/BTC_XMR.pkl
Cached response at ./data/BTC_STRAT.pkl
Cached response at ./data/BTC_ZEC.pkl
Cached response at ./data/BTC_DASH.pkl
Cached response at ./data/BTC_BTS.pkl
Cached response at ./data/BTC_VIA.pkl
Cached response at ./data/BTC_STEEM.pkl
Cached response at ./data/BTC_NAV.pkl


Now we have a dictionary of 9 dataframes, each containing the historical daily average exchange prices between the altcoin and Bitcoin.

We can preview the last few rows of the Ethereum price table to make sure it looks ok.

In [9]:
altcoin_data['ETH'].tail()

,close,high,low,open,quoteVolume,volume,weightedAverage
date,,,,,,,
2018-03-03,0.074500,0.077623,0.074356,0.077562,15058.825646,1139.640375,0.075679
2018-03-04,0.075111,0.077630,0.074389,0.074500,12258.662182,933.480951,0.076149
2018-03-05,0.074020,0.075700,0.073250,0.075277,21500.417584,1606.571070,0.074723
2018-03-06,0.076016,0.076410,0.073787,0.074000,23526.922289,1779.174239,0.075623
2018-03-07,0.073657,0.076090,0.073657,0.076018,8774.041274,655.588755,0.074719


## Merge All Of The Pricing Data Into A Single Dataframe
Next, we will define a simple function to merge a common column of each dataframe into a new combined dataframe.

In [10]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

## Visualize The Pricing Datasets
The next logical step is to visualize how these pricing datasets compare.  For this, we'll define a helper function to provide a single-line command to compare each column in the dataframe on a graph using Plotly.

In [11]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

# Chart the prices

In [12]:
# Merge price of each altcoin into single dataframe Buy
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'weightedAverage')

# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (BTC)', seperate_y_axis=False, y_axis_label='Coin Value (BTC)', scale='log')

# Getting the data with featurese
adds features for classification. First computes the change ratio of all features. This way the features become unitless. Then find the target labels by shifting the weightedAverage and comparing with average value.

In [13]:
def getData(altcoin, predDays, threshold):
    if altcoin == 'Visa':
        df = visa_stock
    elif altcoin == 'BTC':
        df = btc_usd_df
    else:
        # selecting the features of the given altcoin
        df = altcoin_data[altcoin][['open', 'close', 'high', 'low', 'quoteVolume', 'volume', 'weightedAverage']]
        


    # renaming the weightedAverage column
    df = df.rename(columns={'weightedAverage': 'price'})

    df['btc'] = btc_usd_df['weightedAverage']
    

    # disable panda warning
    pd.options.mode.chained_assignment = None  # default='warn'


    
    # adding future days prices as new features
    n = predDays
    df['price+'] = 0
    for i in range(n):
        df['price+'] += (n-i)*df['price'].shift(-i-1)    

    df['price+'] = df['price+'] / ((n+1)*n/2)
#     # remove the last samples since they do not have a tomorow samples
#     df = df.drop(df.index[-n:])

#     df['price+'] = df['price'].shift(-1)
    
#     # remove the last sample since it does not have a tomorow sample
#     df = df.drop(df.index[-1])    
    
    

    df['priceRate+'] = df['price+']/df['price'] - 1

    
    df['vol'] = df['volume']/df['quoteVolume']


    # adding previous 3 days elements as new features
    for i in range(3):
        df['btc-%d'%(i+1)] = df['btc'].shift(i+1)
        df['price-%d'%(i+1)] = df['price'].shift(i+1)
        df['close-%d'%(i+1)] = df['close'].shift(i+1)
        df['open-%d'%(i+1)] = df['open'].shift(i+1)
        df['vol-%d'%(i+1)] = df['vol'].shift(i+1)
        df['totVol-%d'%(i+1)] = df['quoteVolume'].shift(i+1)


    # remove the first n samples since they don't have previous day data
    df = df.drop(df.index[0:3])


    df['btcRate-2'] = df['btc-2']/df['btc-3'] - 1
    df['btcRate-1'] = df['btc-1']/df['btc-2'] - 1
    df['btcRate'] = df['btc']/df['btc-1'] - 1
    df['btcAcc'] = df['btcRate'] - df['btcRate-1']
    df['btcAcc-1'] = df['btcRate-1'] - df['btcRate-2']
    df['btcJerk'] = df['btcAcc'] - df['btcAcc-1']


    df['volRate-2'] = df['vol-2']/df['vol-3'] - 1
    df['volRate-1'] = df['vol-1']/df['vol-2'] - 1
    df['volRate'] = df['vol']/df['vol-1'] - 1
    df['volAcc'] = df['volRate'] - df['volRate-1']
    df['volAcc-1'] = df['volRate-1'] - df['volRate-2']
    df['volJerk'] = df['volAcc'] - df['volAcc-1']


    df['totVolRate-2'] = df['totVol-2']/df['totVol-3'] - 1
    df['totVolRate-1'] = df['totVol-1']/df['totVol-2'] - 1
    df['totVolRate'] = df['quoteVolume']/df['totVol-1'] - 1
    df['totVolAcc'] = df['totVolRate'] - df['totVolRate-1']
    df['totVolAcc-1'] = df['totVolRate-1'] - df['totVolRate-2']
    df['totVolJerk'] = df['totVolAcc'] - df['totVolAcc-1']



    df['closeRate-2'] = df['close-2']/df['close-3'] - 1
    df['closeRate-1'] = df['close-1']/df['close-2'] - 1
    df['closeRate'] = df['close']/df['close-1'] - 1
    df['closeAcc'] = df['closeRate'] - df['closeRate-1']
    df['closeAcc-1'] = df['closeRate-1'] - df['closeRate-2']
    df['closeJerk'] = df['closeAcc'] - df['closeAcc-1']


    df['openRate-2'] = df['open-2']/df['open-3'] - 1
    df['openRate-1'] = df['open-1']/df['open-2'] - 1
    df['openRate'] = df['open']/df['open-1'] - 1
    df['openAcc'] = df['openRate'] - df['openRate-1']
    df['openAcc-1'] = df['openRate-1'] - df['openRate-2']
    df['openJerk'] = df['openAcc']- df['openAcc-1']


    df['priceRate-2'] = df['price-2']/df['price-3'] - 1
    df['priceRate-1'] = df['price-1']/df['price-2'] - 1
    df['priceRate'] = df['price']/df['price-1'] - 1
    df['priceAcc'] = df['priceRate'] - df['priceRate-1']
    df['priceAcc-1'] = df['priceRate-1'] - df['priceRate-2']
    df['priceJerk'] = df['priceAcc'] - df['priceAcc-1']

    df['priceRange'] = df['high']/df['low'] - 1

    df['priceSwing'] = df['close']/df['open'] - 1
    
    
    
    # For adding technical indicators
    raw = df[['open', 'high', 'low', 'close', 'volume']]

    df.head()
    functions = talib.get_functions()
    remove_list = ['MAVP']
    for item in remove_list:
        functions.remove(item)
    


    
    
    data = df[['priceRate+',
                'low', 'high', 'priceRange',
               'open', 'close', 'priceSwing',
               'btc', 'btcRate', 'btcAcc', 'btcJerk',
              'vol', 'volRate', 'volAcc', 'volJerk',
              'price', 'priceRate', 'priceAcc', 'priceJerk',
              'quoteVolume', 'totVolRate', 'totVolAcc', 'totVolJerk']]
    
    
    date = end_date.date()
    dateStr = date.strftime('%Y-%m-%d')
    
    # Google trend interests of the altcoin
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload([altcoin], cat=0, timeframe='2015-01-01 %s'%dateStr, geo='', gprop='')
    interests = pytrends.interest_over_time()
    data[altcoin+'Interest'] = interests[altcoin]
    
    # Google trend interests of the bitcoin
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(['bitcoin'], cat=0, timeframe='2015-01-01 %s'%dateStr, geo='', gprop='')
    interests = pytrends.interest_over_time()
    data['bitcoinInterest'] = interests['bitcoin']
    
    
    # technical indicators
    for func in functions:
        outputs = eval(func)(raw)

        if (type(outputs) is pd.Series):
            data[func] = outputs
    
    n = 150
#     remove the first n samples
    data = data.drop(data.index[0:n])
#     data = data.dropna(axis=0, how='any')

    
    
    
    features = data.drop(['priceRate+'], axis = 1)
    
    
    # getting today information for prediction
    x_today = np.nan_to_num(features.iloc[-1].values)
    
    today = features.iloc[[-1]].index.date
    
    # remove the last samples since they do not have a tomorow samples
    features = features.drop(df.index[-predDays:])
    data = data.drop(df.index[-predDays:])


    X = features.values
    X = np.nan_to_num(X)
#     print('Shape of X: ',X.shape)


    tomorrrowPriceChange = data['priceRate+'].values
    tomorrrowPriceChange = np.nan_to_num(tomorrrowPriceChange)
        
    # finds the median weighted price changes
    priceChangeMed = np.median(tomorrrowPriceChange)
#     print('Median daily price change: %0.4f'%priceChangeMed)

    # computes the target label as whether the tomorrow value exceeds the threshold
    # in order to make the dataset balanced, i.e., number of pos and neg labels be equal
    y = (tomorrrowPriceChange > threshold)*1


#     delta = 0.00
#     y = (np.sign(tomorrrowPriceChange - delta) + np.sign(tomorrrowPriceChange + delta)) / 2.0

#     buyCount = np.sum(y==1)
#     sellCount = np.sum(y==-1)
#     holdCount = np.sum(y==0)

    # print(buyCount)
    # print(sellCount)
    # print(holdCount)
    
    
#     y = (tomorrrowPriceChange > delta)

    return X, y, x_today, today, tomorrrowPriceChange, X.shape[0], priceChangeMed

# Preprocessing

In [14]:
def preprocess(X, x_today, y, testSize, k):
        
    X_train = X[0:-(testSize-1), :]
    y_train = y[0:-(testSize-1)]
    
    # normalize the dataFrame
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X = min_max_scaler.fit_transform(X)
    x_today = min_max_scaler.transform(x_today.reshape(1, -1))
    
    # feature selection
    selector = SelectKBest(mutual_info_classif, k=100)
    X_train = selector.fit_transform(X_train, y_train)
    X = selector.transform(X)
    x_today = selector.transform(x_today)
    
    # dimensionality reduction
    pca = decomposition.PCA(n_components=k)
    pca.fit(X)
    X = pca.transform(X)
    x_today = pca.transform(x_today)
    
    return X, x_today

# Correlation between the best indicators

In [15]:
# correlation_matrix = data.corr()
# plt.figure(figsize=(20,16))
# ax = sns.heatmap(correlation_matrix, vmax=1, square=True,annot=True,cmap='RdYlGn')
# plt.title('Correlation matrix between the features')
# plt.show()

# Model selection

In [16]:
# dict_classifiers = {
#     "Logistic Regression": LogisticRegression(),
#     "Nearest Neighbors": KNeighborsClassifier(),
#     "Linear SVM": SVC(probability=True),
#     "Neural Net": MLPClassifier(alpha = 1),
# #     "Gradient Boosting Classifier": GradientBoostingClassifier(),
# #     "Decision Tree": tree.DecisionTreeClassifier(),
# #     "Random Forest": RandomForestClassifier(n_estimators = 18),
# #     "Naive Bayes": GaussianNB(),
# #     "Bagging-kNN": BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.8)
# #     "AdaBoost": AdaBoostClassifier(n_estimators=100),
# #     "Auto-sklearn": autosklearn.classification.AutoSklearnClassifier(),
# #     "Voting": VotingClassifier(estimators=[
# #                                            ('1',  LogisticRegression()), 
# #                                            ('0', SVC())], voting='hard')
# }

# no_classifiers = len(dict_classifiers.keys())
# def batch_classify(X, y, testSize, verbose = True):
#     df_results = pd.DataFrame(data=np.zeros(shape=(no_classifiers,5)),
#                               columns = ['classifier', 'train_score', 'test_score', 'baseline', 'time'])
#     count = 0
#     for key, classifier in dict_classifiers.items():
#         t_start = time.clock()
#         print(classifier)
#         train_scores = np.zeros(testSize)
#         test_scores = np.zeros(testSize)
#         y_preds = np.zeros(testSize)
#         for i in range(testSize):
#             X_train = X[0:-(i+1), :]
#             y_train = y[0:-(i+1)]
            
#             X_test = X[-(i+1), :].reshape(1, -1)
#             y_test = y[-(i+1)].reshape(1, -1)
            
#             classifier.fit(X_train, y_train)
#             train_scores[i] = classifier.score(X_train, y_train)
#             test_scores[i] = classifier.score(X_test, y_test)
#             y_preds[i] = classifier.predict(X_test)

# #         average_precision = average_precision_score(Y_test, y_pred)
# #         precision = precision_score(Y_test, y_pred)

#         t_end = time.clock()
#         t_diff = t_end - t_start
        
#         df_results.loc[count,'classifier'] = key
#         df_results.loc[count,'train_score'] = np.mean(train_scores)
#         df_results.loc[count,'test_score'] = np.mean(test_scores)
#         df_results.loc[count,'time'] = t_diff
        
# #         df_results.loc[count,'precision'] = precision
# #         df_results.loc[count,'avg_precision'] = average_precision

#         y_test = y[testSize:]
#         baseline = np.sum(y_test==np.argmax([np.sum(y_test==0),
#                                              np.sum(y_test==1)]))/np.size(y_test)
#         df_results.loc[count,'baseline'] = baseline
        
#         count+=1
#     return df_results

In [17]:
# trade = 'ETH'
# testSize = 100
# predDays = 1
# X, y, x_today, today, y_cont, size, median = getData(trade, predDays)

# X, x_today = preprocess(X, x_today, y, testSize, k=20)

# df_results = batch_classify(X, y, testSize)
# display(df_results.sort_values(by='test_score', ascending=False))

# Altcoin selection

In [20]:
testSize = 60
predDays = 1
threshold = 0.01
df_results = pd.DataFrame(data=np.zeros(shape=(len(binance_coins), 7)),
                              columns = ['altcoin', 'days', 'median',
                                         'train_score', 'test_score', 'baseline', 'performance'])

classifier = LogisticRegression()
count = 0
for coin in binance_coins:
    print(coin)
    X, y, x_today, today, y_cont, size, median = getData(coin, predDays, threshold)
    X, x_today = preprocess(X, x_today, y, testSize, k=20)
    
    t_start = time.clock()

    train_scores = np.zeros(testSize)
    test_scores = np.zeros(testSize)
    y_preds = np.zeros(testSize)
    for i in range(testSize):
        X_train = X[0:-(i+1), :]
        y_train = y[0:-(i+1)]

        X_test = X[-(i+1), :].reshape(1, -1)
        y_test = y[-(i+1)].reshape(1, -1)

        classifier.fit(X_train, y_train)
        train_scores[i] = classifier.score(X_train, y_train)
        test_scores[i] = classifier.score(X_test, y_test)
        y_preds[i] = classifier.predict(X_test)

    t_end = time.clock()
    t_diff = t_end - t_start

    df_results.loc[count,'altcoin'] = coin
    df_results.loc[count,'days'] = size
    df_results.loc[count,'median'] = median
    df_results.loc[count,'train_score'] = np.mean(train_scores)
    df_results.loc[count,'test_score'] = np.mean(test_scores)
    df_results.loc[count,'time'] = t_diff


    y_test = y[testSize:]
    baseline = np.sum(y_test==np.argmax([np.sum(y_test==0),
                                         np.sum(y_test==1)]))/np.size(y_test)
    df_results.loc[count,'baseline'] = baseline
    
    df_results.loc[count,'performance'] = (np.mean(test_scores) - baseline)/(1 - baseline)

    count+=1
    
selected_coins = df_results['altcoin'].values[df_results['performance'] > 0.1]

display(df_results.sort_values(by='performance', ascending=False))

ETH
LTC
ETC
XRP
LSK
XMR
STRAT
ZEC
DASH
BTS
VIA
STEEM
NAV


/home/arash/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning:

overflow encountered in reduce

/home/arash/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning:

invalid value encountered in reduce

/home/arash/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:348: RuntimeWarning:

overflow encountered in subtract



,altcoin,days,median,train_score,test_score,baseline,performance,time
6,STRAT,290.0,-0.007292,0.769972,0.78,0.604167,0.444211,0.107928
12,NAV,1008.0,-0.007010,0.650383,0.76,0.606472,0.390133,0.561539
3,XRP,1008.0,-0.005192,0.739152,0.80,0.685804,0.363455,0.522476
10,VIA,1008.0,-0.004247,0.630332,0.74,0.597077,0.354715,0.403372
0,ETH,789.0,-0.002278,0.737634,0.76,0.640054,0.333233,0.393717
9,BTS,1008.0,-0.005718,0.716713,0.76,0.654489,0.305378,0.489576
8,DASH,1008.0,-0.002009,0.722458,0.74,0.643006,0.271696,0.458989
4,LSK,499.0,-0.005099,0.688302,0.70,0.590200,0.267935,0.224780
7,ZEC,342.0,-0.006933,0.720166,0.72,0.636986,0.228679,0.123889
11,STEEM,443.0,-0.009226,0.714932,0.70,0.648855,0.145652,0.228430


# Predictions

In [23]:
testSize = 2
predDays = 1
threshold = 0.01
df_results = pd.DataFrame(data=np.zeros(shape=(len(selected_coins), 5)),
                              columns = ['Altcoin', 'Day', 'Sell', 'Buy', 'Prediction'])

classifier = LogisticRegression()
count = 0
for coin in selected_coins:
    print(coin)
    X, y, x_today, today, y_cont, size, median = getData(coin, predDays, threshold)
    X, x_today = preprocess(X, x_today, y, testSize, k=20)
    

    classifier.fit(X, y)
    probs = classifier.predict_proba(x_today)
    prediction = classifier.predict(x_today)


    df_results.loc[count,'Altcoin'] = coin
    df_results.loc[count,'Day'] = today
    df_results.loc[count,'Sell'] = probs[0][0]
    df_results.loc[count,'Buy'] = probs[0][1]
    df_results.loc[count,'Prediction'] = prediction


    count+=1
    
    
display(df_results.sort_values(by='Buy', ascending=False))

ETH
ETC
XRP
LSK
STRAT
ZEC
DASH
BTS
VIA
STEEM
NAV


/home/arash/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning:

overflow encountered in reduce

/home/arash/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning:

invalid value encountered in reduce

/home/arash/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:348: RuntimeWarning:

overflow encountered in subtract



,Altcoin,Day,Sell,Buy,Prediction
7,BTS,[2018-03-07],0.467505,0.532495,1.0
8,VIA,[2018-03-07],0.591521,0.408479,0.0
10,NAV,[2018-03-07],0.748788,0.251212,0.0
3,LSK,[2018-03-07],0.767270,0.232730,0.0
1,ETC,[2018-03-07],0.769208,0.230792,0.0
5,ZEC,[2018-03-07],0.824135,0.175865,0.0
9,STEEM,[2018-03-07],0.843339,0.156661,0.0
4,STRAT,[2018-03-07],0.869379,0.130621,0.0
2,XRP,[2018-03-07],0.874423,0.125577,0.0
6,DASH,[2018-03-07],0.929102,0.070898,0.0
